In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import random
import copy

from collections import Counter, defaultdict
from deap import base, creator, tools
from tqdm import tqdm

In [2]:
threshold = 0.3 #threshold di discriminazione
egoradius = 2 #Per link da aggiungere - raggio dell'egonetwork
delete = 0.5 #Per link da eliminare - top % con betweeness bassa rimovibili
attr_name = 'gender'

In [3]:
# 0 = Solo aggiunta
# 1 = Solo rimozione
# 2 = Entrambe
mode = 2

In [4]:
fitness = 'nodes'
#marg = marginalization score => somma abs(marg) dei singoli nodi / n.nodi => [0, 1]
#nodes = marginalized nodes

In [5]:
glob = True

n_ran_nodes = 300
n_poss = 250
n_weak = 200

In [6]:
remove_missing = True

# AMARE MI VA- Attribute-aware MARginalization Estimator with MIssing VAlues

In [7]:
g = nx.Graph()
g.name = 'copenhagen'
with open('bt_symmetric.csv') as f:
    for l in f.readlines()[1:]:
        tid, a, b, rssi = l.rstrip().split(',')
        g.add_edge(int(a),int(b), tid=tid)
print('loaded net')

attrs = {n: None for n in g.nodes()} # also fix missing data
with open('genders.csv') as f:
    for l in f.readlines()[1:]:
        node, gender = l.rstrip().split(',')
        attrs[int(node)] = gender
    nx.set_node_attributes(g, attrs, name=attr_name)
print('loaded attributes')

missing = []
for n in attrs:
    if attrs[n] is None:
        missing.append(n)

#g.remove_nodes_from(missing)
#print('unlabeled removed')

loaded net
loaded attributes


In [8]:
print(nx.info(g))

Name: copenhagen
Type: Graph
Number of nodes: 708
Number of edges: 80886
Average degree: 228.4915


In [9]:
attrs = nx.get_node_attributes(g, attr_name)

In [10]:
def random_individual(missing):
    individual = []
    for e in missing:
        individual.append(random.randint(0,1)) #1 = aggiunto/rimosso; 0 = do nothing
    return individual 

In [11]:
def evaluate(individual, g, return_net):
    individual = individual[0] #<- because DEAP
    for node, attr in zip(missing, individual):
        attrs[node] = str(attr)
    
    sizes = dict(Counter(list(attrs.values())))
    sizes['0'] = sizes['0'] / (len(g))
    sizes['1'] = sizes['1'] / (len(g))
    weights = dict(Counter(list(attrs.values())))
    weights['0'] = 1 - sizes['0']
    weights['1'] = 1 - sizes['1']
    
    marg_dict = dict()

    for node in g.nodes():
        attr = attrs[node]

        # COMPUTE MARGINALIZATION
        marg = 0
        egonet = list(g.neighbors(node)) + [node]
        egonet_attrs = [attrs[n] for n in egonet]

        count = dict(Counter(egonet_attrs))[attr]
        size = len(egonet_attrs)

        if size > 2:
            marg = ((count * weights[attr] / (count * weights[attr] + (size-count)* (1 - weights[attr]))) - 0.5) * 2
        else:
            marg = 0

        marg_dict[node] = marg

    disc_nodes = [k for k,v in marg_dict.items() if abs(v) > threshold]

    ov_marg = np.mean([abs(v) for v in marg_dict.values()]) #network marginalization score
    base_marg = [abs(v) for k, v in marg_dict.items() if k not in disc_nodes]
    
    if return_net:        
        print("=== STATS ===")
        print("Marginalized nodes:", len(disc_nodes))
        print("Global Discrimination:", len(disc_nodes) * 100 / len(g.nodes()))
        print("Overall Marginalization Score:", ov_marg)
        print("Marginalization Score w/o Marg nodes:", np.mean(base_marg))
        sns.kdeplot(list(marg_dict.values()))

        return weights, marg_dict, disc_nodes, ov_marg, base_marg
    
    if fitness == 'marg':
        return ov_marg, len(disc_nodes)
    elif fitness == 'nodes':
        return len(disc_nodes), ov_marg



In [12]:
return_net = False #if True, eva function returns the fair network. INSIDE THE GA, IT MUST BE FALSE

creator.create("Fitness", base.Fitness, weights=(-1.0,-1.0)) # <- -1 perché vogliamo minimizzare la fitness
creator.create("Individual", list, fitness=creator.Fitness) #<- l'individuo è definito come lista

toolbox = base.Toolbox() #creiamo il toolbox

toolbox.register("random_individual", random_individual, missing) 
#"nome_della_funzione per deap", nome_della_funzione vera e propria di python, parametri che passi alla funzione

toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.random_individual, n=1) 
# n = numero di individui nella popolazione. Lasciamo 1

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate, g=g, return_net=return_net) #funzione di valutazione. Vedi quanto detto sopra
toolbox.register("mate", tools.cxTwoPoint) #funzione di crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05) #funzione di mutazione custom
toolbox.register("select", tools.selTournament, tournsize=3)
#tools.selNSGA2) #funzione di selezione

In [13]:
if remove_missing:
    g.remove_nodes_from(missing)
    print('unlabeled removed')
    
    attrs = nx.get_node_attributes(g, attr_name)
    
    sizes = dict(Counter(list(attrs.values())))
    sizes['0'] = sizes['0'] / (len(g) + len(missing))
    sizes['1'] = sizes['1'] / (len(g) + len(missing))
    weights = dict(Counter(list(attrs.values())))
    weights['0'] = 1 - sizes['0']
    weights['1'] = 1 - sizes['1']
    
    marg_dict = dict()

    for node in tqdm(g.nodes()):
        attr = attrs[node]

        # COMPUTE MARGINALIZATION
        marg = 0
        egonet = list(g.neighbors(node)) + [node]
        egonet_attrs = [attrs[n] for n in egonet]

        count = dict(Counter(egonet_attrs))[attr]
        size = len(egonet_attrs)

        if size > 2:
            marg = ((count * weights[attr] / (count * weights[attr] + (size-count)* (1 - weights[attr]))) - 0.5) * 2
        else:
            marg = 0

        marg_dict[node] = marg

    disc_nodes = [k for k,v in marg_dict.items() if abs(v) > threshold]
    disc = len(disc_nodes)

    ov_marg = np.mean([abs(v) for v in marg_dict.values()]) #network marginalization score
    base_marg = [abs(v) for k, v in marg_dict.items() if k not in disc_nodes] # marg score w/o marg nodes. used in eva function

else:
    print('Fitness:', fitness)
    NUM_GENERATIONS = 50 #numero di generazioni
    POPULATION_SIZE = 150 #popolazione per gen

    CXPB, MUTPB = 0.5, 0.25 #crossover e mutation probability

    n_HOF = 10 #top soluzioni da ritornare (la "Hall of Fame" di DEAP è il set di tutte le top n soluzioni)

    pop = toolbox.population(n=POPULATION_SIZE)

    hof = tools.HallOfFame(n_HOF)

    stats = tools.Statistics(lambda ind: ind.fitness.values[0])   
    stats.register('min', np.min, axis = 0)
    stats.register('avg', np.mean, axis = 0)

    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + stats.fields

    invalid_individuals = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_individuals)
    for ind, fit in zip(invalid_individuals, fitnesses):
        ind.fitness.values = fit

    hof.update(pop)
    hof_size = len(hof.items)

    record = stats.compile(pop)
    logbook.record(gen=0, best="-", nevals=len(invalid_individuals), **record)
    print(logbook.stream)

    for gen in range(1, NUM_GENERATIONS + 1):

                # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))


        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values


        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        hof.update(offspring)

        # Replace the current population by the offspring
        pop[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(pop) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        print(logbook.stream)


    hof.update(pop) # la HoF è aggiornata con la nuova popolazione (o meglio, i suoi individui migliori w.r.t. fitness)

    plt.figure(1)

    minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
    plt.figure(2)
    sns.set_style("whitegrid")
    #plt.plot(maxFitnessValues, color='red')
    plt.plot(minFitnessValues, color='blue')
    plt.plot(meanFitnessValues, color='green')
    plt.xlabel('Generation')
    if fitness == 'nodes':
        plt.ylabel('Marginalized Nodes')
        plt.title('Avg and Min Marginalized Nodes')
    elif fitness == 'marg':
        plt.ylabel('Marginalization Score')
        plt.title('Avg and Min Marginalization Score')    
    # show both plots:
    plt.show()


    #return hof.items

100%|██████████| 673/673 [00:00<00:00, 42017.34it/s]

unlabeled removed


In [14]:
if remove_missing == False:
    weights, marg_dict, disc_nodes, ov_marg, base_marg = evaluate(hof.items[0], g, True)

In [15]:
if glob == False:
    ran_nodes = random.sample(list(g.nodes()), n_ran_nodes)
    ran_marg = [abs(v) for k, v in marg_dict.items() if k not in ran_nodes]

In [16]:
if mode == 1:
    possible_links = [] 
else:
    plausible = nx.Graph() # stores plausible links

    if glob:
        for node in tqdm(disc_nodes):
            egonet = list(g.neighbors(node)) + [node]
            egonet2 = nx.ego_graph(g, node, center=True, radius=egoradius)
            egonet2.remove_nodes_from(egonet)
            for n in egonet2.nodes():
                if node != n and n in disc_nodes:
                    if marg_dict[node] > 0:
                        if attrs[n] != attrs[node]:
                            plausible.add_edge(node, n)
                    elif marg_dict[node] < 0:
                        if attrs[n] == attrs[node]:
                            plausible.add_edge(node, n)         
                    else:
                        print("ERROR #01")
        possible_links = list(plausible.edges())
        
    else:
        for node in tqdm(ran_nodes):
            egonet = list(g.neighbors(node)) + [node]
            egonet2 = nx.ego_graph(g, node, center=True, radius=egoradius)
            egonet2.remove_nodes_from(egonet)
            for n in egonet2.nodes():
                if node != n:
                    plausible.add_edge(node, n)
        possible_links = random.sample(list(plausible.edges()), n_poss)

100%|██████████| 47/47 [00:11<00:00,  4.25it/s]


In [17]:
if mode == 0:
    weak_links = []
else:
    #Betweenness centrality su archi
    betweenness = nx.edge_betweenness_centrality(g) #dizionario di archi con score di BW centrality

    marg_betweenness = dict()
    
    if glob:
        for k, v in tqdm(betweenness.items()):
            if k[0] in disc_nodes and k[1] in disc_nodes:
                if marg_dict[k[0]] > 0:
                    if attrs[k[0]] == attrs[k[1]]:
                        marg_betweenness[k] = v
                elif marg_dict[k[0]] < 0:
                    if attrs[k[0]] != attrs[k[1]]:
                        marg_betweenness[k] = v
                else:
                    print ("ERROR #02")
        marg_betweenness = dict(sorted(marg_betweenness.items(), key=lambda item: item[1]))
        weak_links = list(marg_betweenness.keys())[:round(len(marg_betweenness) * delete)]
        
    else:
        for k, v in tqdm(betweenness.items()):
            if k[0] in ran_nodes and k[1] in ran_nodes:
                    marg_betweenness[k] = v

        marg_betweenness = dict(sorted(marg_betweenness.items(), key=lambda item: item[1]))
        weak_links = list(marg_betweenness.keys())[:n_weak]

100%|██████████| 75124/75124 [00:00<00:00, 1766087.08it/s]


In [18]:
links = possible_links + weak_links #tutti i links con cui il GA opera

# MARK - MArginalization Reducer using linK

In [19]:
def random_individual(links):
    individual = []
    for e in links:
        individual.append(random.randint(0,1)) #1 = aggiunto/rimosso; 0 = do nothing
    return individual 

In [20]:
def evaluate(individual, g, return_net):
    individual = individual[0] #<- because DEAP
    nodes = 0 #amount of marginalized nodes in fair network
    
    if glob:
        fair_marg = base_marg.copy() #marginalization score of fair network
    else:
        fair_marg = ran_marg.copy()
        
    eva_g = copy.deepcopy(g)
    indexes = [i for i, j in enumerate(individual) if j == 1]
    new_links = [links[i] for i in indexes]

    for l in new_links:
        if l in possible_links:
            eva_g.add_edge(l[0], l[1])
        elif l in weak_links:
            eva_g.remove_edge(l[0], l[1])
            
            
    if glob:
        for node in disc_nodes:
            
            attr = attrs[node] ###aggiunto

            marg = 0
            egonet = list(eva_g.neighbors(node)) + [node]
            egonet_attrs = [attrs[n] for n in egonet]
            count = dict(Counter(egonet_attrs))[attr]
            size = len(egonet)
            if size > 2:
                marg = ((count * weights[attr] / (count * weights[attr] + (size-count)* (1 - weights[attr]))) - 0.5) * 2
                fair_marg.append(abs(marg))
                if abs(marg) > threshold:
                    nodes += 1
    
    else:
        nodes = len(disc_nodes)
        for node in ran_nodes:
            
            
            attr = attrs[node] ###aggiunto

            marg = 0
            egonet = list(eva_g.neighbors(node)) + [node]
            egonet_attrs = [attrs[n] for n in egonet]
            count = dict(Counter(egonet_attrs))[attr]
            size = len(egonet)
            if size > 2:
                marg = ((count * weights[attr] / (count * weights[attr] + (size-count)* (1 - weights[attr]))) - 0.5) * 2
                fair_marg.append(abs(marg))
                if node in disc_nodes:
                    if abs(marg) < threshold:
                        nodes = nodes - 1
                else:
                    if abs(marg) > threshold:
                        nodes = nodes + 1          
        
    budget = sum(individual)
    
    if return_net:
        return nodes, np.mean(fair_marg), budget, eva_g
    
    if fitness == 'nodes':      
        return nodes, budget
    elif fitness == 'marg':
        return np.mean(fair_marg), budget
    else:
        print ("ERROR #03")
        


    #Fitness 1: nodi marginalizzati rimasti
    #Fitness 2: link usati
    
    #A parità di nodi marginalizzati (il meno possibile), la soluzione con meno link usati è la migliore

In [21]:
return_net = False #if True, eva function returns the fair network. INSIDE THE GA, IT MUST BE FALSE

creator.create("Fitness", base.Fitness, weights=(-1.0,-1.0)) # <- -1 perché vogliamo minimizzare la fitness
creator.create("Individual", list, fitness=creator.Fitness) #<- l'individuo è definito come lista

toolbox = base.Toolbox() #creiamo il toolbox

toolbox.register("random_individual", random_individual, links) 
#"nome_della_funzione per deap", nome_della_funzione vera e propria di python, parametri che passi alla funzione

toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.random_individual, n=1) 
# n = numero di individui nella popolazione. Lasciamo 1

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate, g=g, return_net=return_net) #funzione di valutazione. Vedi quanto detto sopra
toolbox.register("mate", tools.cxTwoPoint) #funzione di crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05) #funzione di mutazione custom
toolbox.register("select", tools.selTournament, tournsize=3)
#tools.selNSGA2) #funzione di selezione

/Users/federico/miniforge3/envs/tf/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/federico/miniforge3/envs/tf/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
print('Marg nodes:', len(disc_nodes), '· Marg Score:', ov_marg, '· Available links:', len(links))
print('Fitness:', fitness)
NUM_GENERATIONS = 30 #numero di generazioni
POPULATION_SIZE = 150 #popolazione per gen

CXPB, MUTPB = 0.5, 0.25 #crossover e mutation probability

n_HOF = 10 #top soluzioni da ritornare (la "Hall of Fame" di DEAP è il set di tutte le top n soluzioni)

pop = toolbox.population(n=POPULATION_SIZE)

hof = tools.HallOfFame(n_HOF)

stats = tools.Statistics(lambda ind: ind.fitness.values[0])   
stats.register('min', np.min, axis = 0)
stats.register('avg', np.mean, axis = 0)

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + stats.fields

invalid_individuals = [ind for ind in pop if not ind.fitness.valid]
fitnesses = toolbox.map(toolbox.evaluate, invalid_individuals)
for ind, fit in zip(invalid_individuals, fitnesses):
    ind.fitness.values = fit

hof.update(pop)
hof_size = len(hof.items)

record = stats.compile(pop)
logbook.record(gen=0, best="-", nevals=len(invalid_individuals), **record)
print(logbook.stream)

for gen in range(1, NUM_GENERATIONS + 1):

            # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))


    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1[0], child2[0])
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant[0])
            del mutant.fitness.values


    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Update the hall of fame with the generated individuals
    hof.update(offspring)

    # Replace the current population by the offspring
    pop[:] = offspring

    # Append the current generation statistics to the logbook
    record = stats.compile(pop) if stats else {}
    logbook.record(gen=gen, nevals=len(invalid_ind), **record)
    print(logbook.stream)


hof.update(pop) # la HoF è aggiornata con la nuova popolazione (o meglio, i suoi individui migliori w.r.t. fitness)

plt.figure(1)

minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
plt.figure(2)
sns.set_style("whitegrid")
#plt.plot(maxFitnessValues, color='red')
plt.plot(minFitnessValues, color='blue')
plt.plot(meanFitnessValues, color='green')
plt.xlabel('Generation')
if fitness == 'nodes':
    plt.ylabel('Marginalized Nodes')
    plt.title('Avg and Min Marginalized Nodes')
elif fitness == 'marg':
    plt.ylabel('Marginalization Score')
    plt.title('Avg and Min Marginalization Score')    
# show both plots:
plt.show()


#return hof.items

Marg nodes: 47 · Marg Score: 0.1453457021907056 · Available links: 457
Fitness: nodes
gen	nevals	min	avg    
0  	150   	9  	14.0333
1  	90    	8  	12.3867
2  	91    	6  	11.3067
3  	90    	6  	10.2933
4  	102   	6  	9.33333
5  	97    	6  	8.32   
6  	83    	5  	7.58667
7  	88    	5  	7.14   
8  	93    	5  	6.78   


In [ ]:
best = hof.items[0][0]

In [ ]:
fair_nodes, fair_score, fair_budget, fair_net = evaluate([best], g, True)

print ("Marg Nodes:", fair_nodes, '· Prev:', len(disc_nodes))
print ("Marg Score:", fair_score, '· Prev:', ov_marg)
print ("Links:", fair_net.number_of_edges(), '· Prev:', g.number_of_edges())
if mode == 0:
    print ("Added links:", fair_budget)
elif mode == 1:
    print ("Removed links:", fair_budget)
elif mode == 2:
    print ("Modified links:", fair_budget)
    added = fair_net.number_of_edges() - g.number_of_edges()
    print ("-- of which added:", added)
    print ("-- of which removed:", fair_budget - added)

In [ ]:
print("RandNet - A Random Benchmark for FairNet")
nodes_ran = []
score_ran = []
c = 0
while c < 100:
    ran = random_individual(links)
    fair_nodes, fair_score, fair_budget, fair_net = evaluate([ran], g, True)
    nodes_ran.append(fair_nodes)
    score_ran.append(fair_score)
    c = c+1

print("- Marg nodes")
print("Avg:", np.mean(nodes_ran), "· Min:", min(nodes_ran))
print("- Marg score")
print("Avg:", np.mean(score_ran), "· Min:", min(score_ran))